# Zwischenankunktszeiten

In [1]:
# Datenzugriff
from configs.path_manager import get_path
import pandas as pd

# Utils
from src.utils.production_process import initialize as init
from src.utils.production_process import term as term

In [2]:
# Datei laden
basic_data_path = get_path("data", "basic")
df_instance = pd.read_csv(basic_data_path / "instance.csv")
df_instance

,Routing_ID,Operation,Machine,Processing Time
0,0,0,M00,29
1,0,1,M01,78
2,0,2,M02,9
3,0,3,M03,36
4,0,4,M04,49
...,...,...,...,...
95,9,5,M09,76
96,9,6,M05,47
97,9,7,M03,52
98,9,8,M04,90


## I) Mittlere Zwischenankunftszeit t_a

### 1. Vektor der Bearbeitungszeiten auf der Engpassmaschine

#### a) Bestimmung der Engpassmaschine (7.12)

In [3]:
engpassmaschine = term._get_engpassmaschine(df_instance, verbose = True)
engpassmaschine

Maschinenbelastung (Gesamtverarbeitungszeit):
  M00: 493
  M01: 548
  M02: 556
  M03: 631
  M04: 534
  M05: 416
  M06: 491
  M07: 499
  M08: 531
  M09: 410


'M03'

#### b) Vektor der Bearbeitungszeiten auf der Engpassmaschine erstellen

In [4]:
vec_t_b_mmax = term._get_vec_t_b_mmax(df_instance)
vec_t_b_mmax

[36, 69, 39, 98, 26, 95, 61, 79, 76, 52]

### 2. Job-Wahrscheinlichkeiten

In [18]:
# Anzahl unterschiedlicher Jobs
n_routings = df_instance['Routing_ID'].nunique()

p = [1.0 / n_routings] * n_routings
p

[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]

### 3. Berechne die mittlere Zwischenankunftszeit t_a nach Formel (7.16) 
> All-in-One

In [8]:
t_a = term.calculate_mean_interarrival_time(df_instance, u_b_mmax = 0.9)
t_a

70.11

## II) Ankunftszeiten

#### Generierung exponentiell verteilte Zwischenankunftszeiten mit Mittelwert t_a 
#### mit eine zufällige Job-Reihenfolge

In [17]:
df_jssp = init.generate_jobs_from_routings(df_instance, shuffle = False)
df_jobs = df_jssp[["Job", "Routing_ID"]].drop_duplicates()

arrivals = term.generate_arrivals_from_mean_interarrival_time(10, t_a)
df_jobs_arrivals = df_jobs.copy()
df_jobs_arrivals['Arrival'] = arrivals
df_jobs_arrivals

,Job,Routing_ID,Arrival
0,0,0,11
10,1,1,96
20,2,2,118
30,3,3,160
40,4,4,197
50,5,5,263
60,6,6,324
70,7,7,341
80,8,8,356
90,9,9,441
